In [13]:
from taskgen import Function, Agent, Memory
import json
import os
from dotenv import load_dotenv

# Specify the path to your .env file
dotenv_path = '../../../.env'

# Load environment variables from a .env file
load_dotenv()

# Now you can safely use the environment variable
api_key = os.getenv('OPENAI_API_KEY')

In [14]:
def store_name(shared_variables, name: str):
    shared_variables['User Information'] = name
    return {'Output': f'Nice to meet you, {name}!'}

def greet_user(shared_variables):
    stored_name = shared_variables['User Information']
    return {'Output': f'Hello {stored_name}! How can I assist you today ?'}

def ingest_json(memory_bank, path: str):
    with open(path, 'r') as file:
        json_data = file.read()
    documents = json.loads(json_data)
    memory_bank['Documents'] = Memory(documents, top_k=3, mapper=lambda doc: doc['content'])
    return {'Output': 'Data has been stored successfully'}

def access_relevant_documents(memory_bank, query: str):
    relevant_documents = memory_bank['Documents'].retrieve_by_ranker(query)
    document_names = [doc['name'] for doc in relevant_documents]
    return {'Output': f'The most relevant documents for "{query}" are: {", ".join(document_names)}'}

In [15]:
# Define functions for the conversational agent
fn_list = [
    Function(fn_description='Greets the user', external_fn=greet_user, output_format={'Output': 'Greeting message'}),
    Function(fn_description='Stores the <name> in memory', external_fn=store_name, output_format={'Output': 'Confirmation message'}),
    Function(fn_description='Output a sentence with words <var1> and <var2> in the style of <var3>', 
                     output_format = {'output': 'sentence'}),
    Function(fn_description='Ingest JSON data from <path> and store it in memory', fn_params=[{'name': 'path', 'type': 'str'}], external_fn=lambda shared_variables, path: ingest_json(conversational_agent.memory_bank, path), output_format={'Output': 'Confirmation message'}),
    Function(fn_description='Access relevant documents based on a <query>', fn_params=[{'name': 'query', 'type': 'str'}], external_fn=lambda shared_variables, query: access_relevant_documents(conversational_agent.memory_bank, query), output_format={'Output': 'Relevant document names'})
]

# Create the conversational agent
conversational_agent = Agent('Conversational Agent', 'Engages in personalized conversations with users', 
                             memory_bank={'Documents': Memory(), 'Function': Memory()},
                             shared_variables={'User Information': ''}).assign_functions(fn_list)

conversational_agent.memory_bank['Function'] = Memory(fn_list, top_k=3, mapper=lambda f: f.fn_description)



In [ ]:
user_input = input('User: ')
# see the auto-generated names of your functions :)
conversational_agent.list_functions()

In [ ]:
# Store the user's name in memory
output = conversational_agent.run(f'Store the {user_input} in memory')
print(f'Agent: {output}')

# visualise the shared variables
print('Shared Variables:', conversational_agent.shared_variables)

In [ ]:
# Run the conversational agent
output = conversational_agent.run('Greet the user')
print(f'Agent: {output}')

In [16]:
path = 'data/documents.json'

# Ingest the JSON data and store it in memory
output = conversational_agent.run(f'Ingest JSON data from "{path}" and store it in memory')
print(f'Agent: {output}')

query = input('User: ')
# Access relevant documents based on the user's query
output = conversational_agent.run(f'Access relevant documents based on a "{query}"')
print(f'Agent: {output}')

Filtered Function Names: store_name, ingest_JSON_data, access_documents
Subtask identified: Ingest JSON data from "data/documents.json" and store it in memory
Calling function ingest_JSON_data with parameters {'path': 'data/documents.json'}
> {'Output': 'Data has been stored successfully'}

Filtered Function Names: store_name, ingest_JSON_data, access_documents
Task completed successfully!

Agent: [{'Output': 'Data has been stored successfully'}]
Filtered Function Names: sentence_with_style, ingest_JSON_data, access_documents
Subtask identified: Access relevant documents based on the query "When will the company implement the two-day work in office policy?"
Calling function access_documents with parameters {'query': 'When will the company implement the two-day work in office policy?'}
> {'Output': 'The most relevant documents for "When will the company implement the two-day work in office policy?" are: April Work From Home Update, Wfh Policy Update May 2023, Work From Home Policy'}

Fi